## Synonym Learning Test

In [10]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
from gensim.models.doc2vec import TaggedLineDocument
import multiprocessing

###### A Simple file with country to capital relations.I manually copied some mentions of Islamabad  and duplicated them, I also  replaced Islmabad with Islo in the duplicates.

In [11]:
documents = TaggedLineDocument('capital.txt')

## Preprocessing
To set the same vocabulary size with original paper. We first calculate the optimal **min_count** parameter.

In [15]:
pre = Doc2Vec(min_count=0)
pre.scan_vocab(documents)

AttributeError: 'Doc2Vec' object has no attribute 'scan_vocab'

In [4]:
for num in range(0, 20):
    print('min_count: {}, size of vocab: '.format(num), pre.scale_vocab(min_count=num, dry_run=True)['memory']['vocab']/700)

('min_count: 0, size of vocab: ', 540)
('min_count: 1, size of vocab: ', 540)
('min_count: 2, size of vocab: ', 533)
('min_count: 3, size of vocab: ', 533)
('min_count: 4, size of vocab: ', 533)
('min_count: 5, size of vocab: ', 533)
('min_count: 6, size of vocab: ', 533)
('min_count: 7, size of vocab: ', 533)
('min_count: 8, size of vocab: ', 533)
('min_count: 9, size of vocab: ', 533)
('min_count: 10, size of vocab: ', 533)
('min_count: 11, size of vocab: ', 532)
('min_count: 12, size of vocab: ', 532)
('min_count: 13, size of vocab: ', 532)
('min_count: 14, size of vocab: ', 532)
('min_count: 15, size of vocab: ', 532)
('min_count: 16, size of vocab: ', 532)
('min_count: 17, size of vocab: ', 532)
('min_count: 18, size of vocab: ', 532)
('min_count: 19, size of vocab: ', 532)


In the original paper, they set the vocabulary size 915,715. It seems similar size of vocabulary if we set min_count = 19. (size of vocab = 898,725)

## Training the Doc2Vec Model
To train Doc2Vec model by several method, DBOW and DM, we define the list of models.

In [13]:
cores = multiprocessing.cpu_count()

models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, size=200, window=8, min_count=1, iter=10, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, size=200, window=8, min_count=1, iter =10, workers=cores),
]

In [14]:
models[0].build_vocab(documents)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

........call logger .............
scanning vocabs
finishing scan
 ................................ making the cum_table ......................................................... 



TypeError: argument of type 'NoneType' is not iterable

Now we’re ready to train Doc2Vec of the English Wikipedia. 

In [7]:
for model in models:
    %%time model.train(documents, total_examples=model.corpus_count, epochs=model.iter)

CPU times: user 15.3 s, sys: 2.62 s, total: 17.9 s
Wall time: 11.6 s
CPU times: user 13 s, sys: 2.5 s, total: 15.5 s
Wall time: 11.4 s


## Similarity interface

In [8]:
for model in models:
    print(str(model))
    #inferred_docvec = model.infer_vector("Islamabad", alpha=0.1, min_alpha=0.0001, steps=5000)
    #pprint(model.most_similar_cosmul(positive=[inferred_docvec], topn=20))
    pprint(model.similar_by_word("Islamabad", topn=10, restrict_vocab=None))
    #pprint(model.similarity("Islamabad", "Islo"))
  
    #pprint(model.similarity("Islamabad", "Islo"))
    
    print "................................................................."

Doc2Vec(dbow+w,d200,n5,w8,s0.001,t4)
[(u'Islo', 0.8855158090591431),
 (u'Indonesia', 0.7330083847045898),
 (u'Uganda', 0.7254679799079895),
 (u'Kathmandu', 0.7204177379608154),
 (u'Hanoi', 0.7183682918548584),
 (u'Sudan', 0.7121343016624451),
 (u'Helsinki', 0.7099384069442749),
 (u'Havana', 0.7086778879165649),
 (u'Khartoum', 0.6969285607337952),
 (u'Madrid', 0.6860840320587158)]
.................................................................
Doc2Vec(dm/m,d200,n5,w8,s0.001,t4)
[(u'Islo', 0.946414589881897),
 (u'Uganda', 0.8783252239227295),
 (u'Jakarta', 0.8753033876419067),
 (u'Kathmandu', 0.874061107635498),
 (u'Khartoum', 0.8719033598899841),
 (u'Indonesia', 0.8711293935775757),
 (u'Sudan', 0.8705962300300598),
 (u'Afghanistan', 0.8677237033843994),
 (u'Rwanda', 0.8675500154495239),
 (u'Nepal', 0.8663867712020874)]
.................................................................


### Even though the synonym appeared only 33 times its still closer to the word than rest of the dictionay

In [9]:
print model.wv.vocab["Islo"]
print model.wv.vocab["Islamabad"]
print model.wv.vocab["Rwanda"]

Vocab(count:33, index:745, sample_int:4294967296)
Vocab(count:122, index:119, sample_int:4294967296)
Vocab(count:78, index:318, sample_int:4294967296)


In [10]:
distance = model.wmdistance("islamabad", "Islamabad ")

ImportError: Please install pyemd Python package to compute WMD.